In [ ]:
!pip install langchain-community
!pip install langchain-pinecone
!pip install langchain-openai

### 1. 기본 RAG

In [ ]:
import os
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore


os.environ['PINECONE_API_KEY'] = ""
os.environ['OPENAI_API_KEY'] = ""

index_name = "playground" #qna-rag"
embeddings = OpenAIEmbeddings()

vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)
retriever = vectorstore.as_retriever()

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""
rag_qna_prompt_template = PromptTemplate.from_template(prompt)

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 사용자의 입력이 RunnablePassthrough 로 넘어감
# https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.passthrough.RunnablePassthrough.html
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | rag_qna_prompt_template
    | llm
    | StrOutputParser()
)

In [ ]:
rag_chain.invoke("미성년자도 판매 회원 등록이 가능한가요?")

### 2. Chat History 기반 RAG

In [ ]:
### Query Rewriting ###
# 유저 : 미성년자도 판매 회원 등록이 가능한가요?
# 챗봇 : 스마트스토어는 만 14세 미만의 개인(개인 사업자 포함) 또는 법인사업자는 입점이 불가함을 양해 부탁 드립니다.
# 유저 : 저는 만 18세입니다.
# ==> "만 18세인 경우, 판매 회원 등록이 가능한가요?"

from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [ ]:
from langchain_core.messages import HumanMessage

chat_history = []

while True:
    question = input("Ask a question: ")
    if question == "exit":
        break
    ai_msg = history_aware_retriever.invoke({"input": question, "chat_history": chat_history})
    chat_history.extend([HumanMessage(content=question), ai_msg["answer"]])
    print(ai_msg["answer"])


### Query Rewriting ###
# 유저 : 미성년자도 판매 회원 등록이 가능한가요?
# 챗봇 : 스마트스토어는 만 14세 미만의 개인(개인 사업자 포함) 또는 법인사업자는 입점이 불가함을 양해 부탁 드립니다.
# 유저 : 저는 만 18세입니다.
# ==> "만 18세인 경우, 판매 회원 등록이 가능한가요?"


In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""

history_based_rag_qna_prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, history_based_rag_qna_prompt_template)

history_based_rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [ ]:
from langchain_core.messages import HumanMessage

chat_history = []

while True:
    question = input("Ask a question: ")
    if question == "exit":
        break
    ai_msg = history_based_rag_chain.invoke({"input": question, "chat_history": chat_history})
    chat_history.extend([HumanMessage(content=question), ai_msg["answer"]])
    print(ai_msg["answer"])


### Query Rewriting ###
# 유저 : 미성년자도 판매 회원 등록이 가능한가요?
# 챗봇 : 스마트스토어는 만 14세 미만의 개인(개인 사업자 포함) 또는 법인사업자는 입점이 불가함을 양해 부탁 드립니다.
# 유저 : 저는 만 18세입니다.
# ==> "만 18세인 경우, 판매 회원 등록이 가능한가요?"